In [1]:
!pip install chromadb pypdf openai langchain 

In [3]:
!pip install sentence-transformers

In [88]:
import numpy as np
import chromadb
import pandas as pd
from pypdf import PdfReader
import numpy as np
from dotenv import load_dotenv
import os 
from openai import OpenAI
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_SECRET_KEY")

In [9]:
#provided by instructor 
def project_embeddings(embeddings, umap_transform):
    """
    Projects the given embeddings using the provided UMAP transformer.

    Args:
    embeddings (numpy.ndarray): The embeddings to project.
    umap_transform (umap.UMAP): The trained UMAP transformer.

    Returns:
    numpy.ndarray: The projected embeddings.
    """
    projected_embeddings = umap_transform.transform(embeddings)
    return projected_embeddings


def word_wrap(text, width=87):
    """
    Wraps the given text to the specified width.

    Args:
    text (str): The text to wrap.
    width (int): The width to wrap the text to.

    Returns:
    str: The wrapped text.
    """
    return "\n".join([text[i : i + width] for i in range(0, len(text), width)])


def extract_text_from_pdf(file_path):
    """
    Extracts text from a PDF file.

    Args:
    file_path (str): The path to the PDF file.

    Returns:
    str: The extracted text.
    """
    text = []
    with open(file_path, "rb") as f:
        pdf = PdfReader(f)
        for page_num in range(pdf.get_num_pages()):
            page = pdf.get_page(page_num)
            text.append(page.extract_text())
    return "\n".join(text)


def load_chroma(filename, collection_name, embedding_function):
    """
    Loads a document from a PDF, extracts text, generates embeddings, and stores it in a Chroma collection.

    Args:
    filename (str): The path to the PDF file.
    collection_name (str): The name of the Chroma collection.
    embedding_function (callable): A function to generate embeddings.

    Returns:
    chroma.Collection: The Chroma collection with the document embeddings.
    """
    # Extract text from the PDF
    text = extract_text_from_pdf(filename)

    # Split text into paragraphs or chunks
    paragraphs = text.split("\n\n")

    # Generate embeddings for each chunk
    embeddings = [embedding_function(paragraph) for paragraph in paragraphs]

    # Create a DataFrame to store text and embeddings
    data = {"text": paragraphs, "embeddings": embeddings}
    df = pd.DataFrame(data)

    # Create or load the Chroma collection

    collection = chromadb.Client().create_collection(collection_name)

    # Add the data to the Chroma collection
    for ids, row in df.iterrows():

        collection.add(ids=ids, documents=row["text"], embeddings=row["embeddings"])
        # collection.add(text=row["text"], embedding=row["embeddings"])

    return collection


In [11]:
#extracting the text from the pdf 
reader = PdfReader("data/microsoft-annual-report.pdf")
#extracting the text from the pdf 
pdf_texts = [p.extract_text().strip() for p in reader.pages]

In [13]:
#splitting the text into chunks using langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
#doing this helps it make more readable/structured
character_splitter = RecursiveCharacterTextSplitter(
                                                    #the seperators tell where to split
                                                    #there is an order of prefernce
                                                    #first try at \n, then at \n\n, then at . 
                                                    separators = ["\n", "\n\n", ". ", " ", ""], 
                                                    chunk_size = 1000, 
                                                    chunk_overlap = 0)
combo = "\n\n".join(pdf_texts)
character_split_texts = character_splitter.split_text(combo)

In [15]:
#now we want to split the text into chunks of about 256 tokens 
#making the object that will split the text by tokens 
token_splitter = SentenceTransformersTokenTextSplitter(tokens_per_chunk = 260, chunk_overlap = 0)

#actually splitting by tokens 
#doing this for each textsplit 
token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)
    

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\jesun\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jesun\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
#begin the embedding process 
import chromadb 
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
#instantiate the embedding function 
embedding_function = SentenceTransformerEmbeddingFunction()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\jesun\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jesun\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [46]:
#instantiate the chroma client 
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection(
    #name the function
     "microsoft-collection",
     #do the embedding 
     embedding_function = embedding_function)
#extract the embeddings 
ids = [str(i) for i in range(len(token_split_texts))]
#now let us add the embeddings to the database 
chroma_collection.add(ids = ids, documents = token_split_texts) 


InternalError: Collection [microsoft-collection] already exists

In [56]:
#the question
query = "What was the total revenue for the year?" 
#querying the collection and finding appropriate documents 
results = chroma_collection.query(query_texts=[query], n_results = 5)

#getting the 5 most relevant documents 
retrieved_documents = results["documents"][0]


In [90]:
#function for generation a response 
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])  
def augment_query_generated(query, model="gpt-3.5-turbo"):
    prompt = """You are a helpful expert financial research assistant. 
   Provide an example answer to the given question, that might be found in a document like an annual report."""
    messages = [
        {
            "role": "system",
            "content": prompt,
        },
        {"role": "user", "content": query},
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [92]:
original_query = "What was the total profit for the year, and how does it compare to the previous year?"
hypothetical_answer = augment_query_generated(original_query)

#now we want to combine the original query and the hypothetical answer 
joint_query = f"{original_query}{hypothetical_answer}"

In [96]:
#now we want to requery the collection, with this updated query that contains both the original and the hallucinated answer
results = chroma_collection.query(
    query_texts = joint_query, 
    n_results = 5,
    #we want to include the embeddings also so it is possible to create a graph that show this visually 
    include = ["documents", "embeddings"])